# Tarea 1: Visión Computacional

ME4707 - Robótica - Semestre 2021-1

Profesor: Juan C. Zagal - Auxiliar: Cristián Herrera - Laboratorios: Raimundo Lorca

# Instalación
## 1. Google Colab
En esta tarea, que se trabajará durante los primeros cuatro laboratorios computacionales, se utilizará Python 3 y la librería de visión computacional OpenCV. La forma más fácil de tener un ambiente de desarrollo de Python con todas las bibliotecas más comunes es utilizar **Google Colaboratory**. Colaboratory es un entorno Jupyter notebook gratuito que se ejecuta completamente en la nube. Puede escribir y ejecutar código, y solo requiere de una cuenta google.

https://colab.research.google.com/notebooks/welcome.ipynb

Debe trabajar sobre este mismo archivo .ipynb completando lo que se solicita en cada problema.

## 2. OpenCV
En esta tarea se utilizará principalmente **OpenCV**. La Open Surce Computer Vision es una librería especializada en herramientas de visión computacional y en todo lo que respecta a visión artificial en general. De este modo provee de funcionalidades de todo tipo de complejidad, desde operaciones básicas de procesamiento de imágenes, hasta algoritmos de reconocimiento de objetos.

<img src="https://raw.githubusercontent.com/cherrerab/roboticafcfm/master/auxiliar_01/bin/opencv_logo.png" height="200">

Por supuesto, esta librería cuenta con su documentación correspondiente. En esta puede encontrar descripciones más detalladas de sus funcionalidades, así como también ejemplos, tutoriales y otros.

https://docs.opencv.org/master/


##3. Imágenes

Para facilitar la carga de los archivos necesarios para completar la tarea, se ha habilitado un directorio en el github del curso `roboticafcfm`. Para clonar el repositorio en el entorno de Colab solo debe ejecutar el siguiente bloque de código.

In [ ]:
!git clone https://github.com/cherrerab/roboticafcfm.git
%cd /content/roboticafcfm

# Problema 1: Image Masking
Como se ha visto en las clases del curso, las imágenes digitales consisten en arreglos discretos de valores numéricos, donde cada valor define la intensidad de iluminación que posee un punto `pixel` específico en la imagen.

Por ejemplo, cargue la imagen `balls.jpg` e imprima las dimensiones de esta. Esto lo puede hacer mediante el siguiente código:

In [ ]:
# el módulo matplotlib.plt nos permite visualizar las imágenes
# el módulo numpy contiene herramientas para arreglos numéricos
from matplotlib import pyplot as plt
import numpy as np
import cv2

# para leer una imagen, hay que ingresar su ubicación en la función cv2.imread
img = cv2.imread('tarea_01//images//balls.jpg')

# las dimensiones de la imagen pueden ser obtenidas utilizando el atributo .shape
print('la imagen balls.jpg tiene dimensiones: ', img.shape )

Se puede notar que la imagen es un arreglo o bien, matriz de tres dimensiones `640x640x3`. En este caso, el primer valor corresponde a la altura `height` de la imagen, el segundo al ancho `width` de esta, y el último a la cantidad de canales que posee. Así, se trata de una imagen de 640x640 de 3 canales RGB.

Ahora, al cargar imágenes utilizando cv2.imread estas son cargadas como BGR y no RGB, por lo que los canales están inicialmente mal asignados. Para corregir esto existe la función `cv2.cvtColor` con la cual es posible cambiar el colorspace de la imagen.

In [ ]:
# leer imagen
img = cv2.imread('tarea_01//images//balls.jpg')

# mostrar imagen en BGR
fig = plt.figure(figsize=(6, 6))
plt.imshow(img)
plt.axis('off')

# corregir a RGB
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# mostrar imagen en RGB (correcta)
fig = plt.figure(figsize=(6, 6))
plt.imshow(img_rgb)
plt.axis('off')

# también se puede convertir a escala de grises
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
fig = plt.figure(figsize=(6, 6))
plt.imshow(img_gray, cmap='gray')
plt.axis('off')

Como se puede ver, la imagen consiste principalmente en tres bolas de luz (dos rojas y una de color verde). No obstante, existe bastante contenido adicional que no parece muy relevante y que podría ser removido. Así, se podría procesar la imagen de tal manera de aislar las bolas tal como se muestra en la imagen `balls_masked.png`.

<img src="https://raw.githubusercontent.com/cherrerab/roboticafcfm/master/tarea_01/bin/prob_01.png" height="210">

Para lograr lo anterior se puede generar una máscara `mask` a partir de una imagen binarizada y multiplicar esta con la imagen original.

En términos simples, una `mask` no es más que que un arreglo binario que describe las zonas de interés de una imagen, y por tanto, permite realizar segmentaciones como la mostrada a continuación.

<img src="https://raw.githubusercontent.com/cherrerab/roboticafcfm/master/tarea_01/bin/prob_02.png" height="250">

Una forma de binarizar una imagen es utilizando la función `cv2.inRange(img, umbral, 255)`, donde `umbral` indica el valor que define la segementación del resultado. También se puede utilizar la función `cv2.threshold` que podrá encontrar en la documentación.

In [ ]:
# ---
# leer imagen
img = cv2.imread('tarea_01//images//balls.jpg')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# convertir a escala de grises
img_gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

fig = plt.figure(figsize=(6, 6))
plt.imshow(img_gray, cmap='gray')
plt.axis('off')

# ---
# binarizar con cv2.inRange
# **los valores de la máscara deben estar entre 0 y 1
mask = cv2.inRange(img_gray, 50, 255)
mask = mask/255

fig = plt.figure(figsize=(6, 6))
plt.imshow(mask, cmap='gray')
plt.axis('off')

# ---
# multiplicar máscara con imagen original

# inicializar nueva imagen img_mask
img_mask = np.zeros_like(img)

# para cada uno de los canales en img
for c in range(3):
  img_mask[:,:,c] = np.multiply(img[:,:,c], mask)

fig = plt.figure(figsize=(6, 6))
plt.imshow(img_mask)
plt.axis('off')


En el ejemplo anterior no se lograron aislar por completo las bolas de luz debido a que el umbral seleccionado en la binarización no es el adecuado. Un método para determinar este umbral es analizando el histograma de la imagen.

Los histogramas de una imagen, corresponden a la representación gráfica de la distribución de las intensidades al interior de la imagen. Estos pueden ser generados utilizando la función `plt.hist(array, bins)`, donde el parámetro `array` consiste en el `np.array` a procesar, mientras que `bins` determina la discretización del histograma.

(a) Convierta la imagen a escala de grises y visualice su histograma (cree una nueva figura para esto y utilice la función `plt.hist` sobre `img.flatten()`). A partir del histograma, determine el umbral adecuado para la máscara y genere la imagen con las esféras aisladas.

In [ ]:
# RESPUESTA PROBLEMA 1.a

# 1. leer la imagen (RGB)

# 2. convertir a escala de grises

# 3. plotear histograma de la imagen en escala de grises

# 4. crear máscara binaria a partir del umbral del histograma

# 5. multiplicar imagen original con máscara y visualizar


b) Suponga que ahora solo desea segmentar la esfera de color verde. Sobre la imagen RGB elija el canal correspondiente y obtenga una nueva máscara para cumplir el requisito anterior.

<img src="https://raw.githubusercontent.com/cherrerab/roboticafcfm/master/tarea_01/bin/prob_03.png" height="250">

In [ ]:
# RESPUESTA PROBLEMA 1.b

# 1. leer la imagen (RGB)

# 2. seleccionar canal de la imagen

# 3. plotear histograma del canal

# 4. crear máscara binaria a partir del umbral del histograma

# 5. multiplicar imagen original con máscara y visualizar

# Problema 2: Applying Numerical Functions

Parte importante en el procesamiento de imágenes digitales es la aplicación de transformaciones (lineales y no-lineales) sobre los valores numéricos de estas. Para introducir esta idea, considere el siguiente problema sobre la imagen en escala de grises `old.jpg`.

In [ ]:
# leer imagen
img = cv2.imread('tarea_01//images//old.jpg')
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# mostrar imagen
fig = plt.figure(figsize=(6, 6))
plt.imshow(img, cmap='gray', vmin=0, vmax=255)
plt.axis('off')


Como se puede observar, esta imagen posee un muy bajo contraste, donde claramente los valores de los pixeles no utilizan todo el rango de intensidades `[0, 255]` disponible. Para corregir esto, se puede aplicar una transformación lineal punto a punto.

<img src="https://raw.githubusercontent.com/cherrerab/roboticafcfm/master/tarea_01/bin/prob_04.png" height="250">

(a) Analice el histograma de la imagen y determine el rango de intensidades contenido en esta (obtenga los valores `min` y `max` correspondientes). Luego, considerando estos límites (`v_min` y `v_max`), aplique la transformación `scaling`:

`new_px = 255*(px - v_min)/(v_max - v_min)`

sobre cada uno de los pixeles en la imagen y visualice el resultado. Esta función `scaling` simplemente escala los valores de los pixeles tal que el mínimo y el máximo correspondan a 0 y 255, respectivamente.

**Al operar numéricamente los valores de una imagen `np.uint8`, suele convenir transformar esta a valores punto flotante `np.float32`.

In [ ]:
# RESPUESTA PROBLEMA 2.a

# ---
# leer imagen
img = cv2.imread('tarea_01//images//old.jpg')
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# transformar imagen a np.float32


# ---
# plotear histograma de la imagen


# defina valores v_min y v_max (puede utilizar np.min y np.max si lo prefiere)
v_min = 
v_max = 

# ---
# transformar imagen
for i in range(img.shape[0]):
  for j in range(img.shape[1]):

    # aplicar transformación
    img[i, j] = 

# transformar imagen a np.uint8


# mostrar imagen
fig = plt.figure(figsize=(6, 6))
plt.imshow(img, cmap='gray', vmin=0, vmax=255)
plt.axis('off')

(b) Siguiendo el mismo procedimiento, normalice los valores de intensidad de la imagen entre `[0, 1]` y luego aplique la transformación:

`new_px = log( px + alpha )`

sobre la imagen. Puede que sea necesario aplicar un último `scaling` al rango `[0, 255]` posterior a la transformación.

¿Cuales son los efectos de esta operación? ¿Por qué se necesita el parámetro  `alpha` y como afecta este los resultados? ¿Cual es
un valor adecuado para `alpha`?




In [ ]:
# RESPUESTA PROBLEMA 2.b

# leer imagen
img = cv2.imread('tarea_01//images//old.jpg')
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
height, width, _ = img.shape

# transformar imagen a np.float32


# plotear histograma de la imagen


# ---
# normalizar imagen (scaling entre 0 y 1)
v_min = 
v_max = 

for i in range(height):
  for j in range(width):
    img[i, j] = 

# ---
# transformar imagen
alpha = 0.05

for i in range(height):
  for j in range(width):
    img[i, j] = 


# plotear histograma de la imagen transformada
plt.hist(img.flatten(), 40)


# ---
# escalar imagen (scaling entre 0 y 255)
v_min = 
v_max = 

for i in range(height):
  for j in range(width):
    img[i, j] = 


# transformar imagen a np.uint8


# mostrar imagen
fig = plt.figure(figsize=(6, 6))
plt.imshow(img, cmap='gray')
plt.axis('off')

# Problema 3: Coin Classification

Considere la imagen de monedas `coins.png`. Se desea elaborar un algoritmo que permita clasificar cada una de las monedas de acuerdo a su tamaño.

<img src="https://raw.githubusercontent.com/cherrerab/roboticafcfm/master/tarea_01/bin/prob_05.png" height="250">

En el Problema 1 de esta tarea ya se vio como es posible segmentar los cuerpos de una imagen mediante la binarización adecuada de esta.

In [ ]:
# leer imagen
img = cv2.imread('tarea_01//images//coins.png')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# convertir a escala de grises
img_gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

fig = plt.figure(figsize=(6, 6))
plt.imshow(img_gray, cmap='gray')
plt.axis('off')

# binarizar con cv2.inRange (los valores de la máscara deben estar entre 0 y 1)
mask = cv2.inRange(img_gray, 90, 255)
mask = mask/255

fig = plt.figure(figsize=(6, 6))
plt.imshow(mask, cmap='gray')
plt.axis('off')

No obstante, esto no permite aislar cada una de las monedas por si sola ya que estas son relativamente del mismo color (o más bien, tonalidad de gris). Para resolver esto, se puede utilizar el algoritmo `Connected Componentes` (implementado en `cv2.connectedComponents`) el cual, a partir de una imagen binarizada etiqueta cada uno de los cuerpos presentes. Esto último permite acceder a cada uno de los cuerpos mediante su etiqueta o bien, índice.

<img src="https://raw.githubusercontent.com/cherrerab/roboticafcfm/master/tarea_01/bin/prob_06.png" height="250">

In [ ]:
# ---
# leer imagen
img = cv2.imread('tarea_01//images//coins.png')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# convertir a escala de grises
img_gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

# binarizar con cv2.inRange (los valores de la máscara deben estar entre 0 y 1)
mask = cv2.inRange(img_gray, 90, 255)/255
mask = np.uint8(mask)

fig = plt.figure(figsize=(6, 6))
plt.imshow(mask, cmap='gray')
plt.axis('off')

# ---
# etiquetar mediante connected components
# retorna cantidad de cuerpos identificados y una imagen con las etiquetas
num_coins, labels = cv2.connectedComponents(mask)

print('monedas identificadas: {:d}'.format((num_coins - 1)))

fig = plt.figure(figsize=(6, 6))
plt.imshow(labels, cmap='jet')
plt.axis('off')

# obtener mask de moneda con etiqueta 6
coin_6 = np.uint8( labels==6 )

fig = plt.figure(figsize=(6, 6))
plt.imshow(coin_6, cmap='gray')
plt.axis('off')


(a) Utilizando un `for loop` itere sobre los índices de las monedas y clasifiquelas a partir de su tamaño (note que puede hacer esto a partir de su área o cantidad de piexeles), genere una imagen binaria con las monedas grandes y otra con las monedas pequeñas.

<img src="https://raw.githubusercontent.com/cherrerab/roboticafcfm/master/tarea_01/bin/prob_07.png" height="250">

In [ ]:
# RESPUESTA PROBLEMA 3.a

# leer imagen
img = cv2.imread('tarea_01//images//coins.png')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# convertir a escala de grises
img_gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

# binarizar con cv2.inRange (los valores de la máscara deben estar entre 0 y 1)
mask = cv2.inRange(img_gray, 90, 255)/255
mask = np.uint8(mask)

# etiquetar mediante connected components
num_coins, labels = 

# inicializar imágenes de monedas pequeñas y grandes
small_coins = np.zeros_like(mask)
big_coins = np.zeros_like(mask)

# para cada una de las etiquetas identificadas
for i in np.arange(num_coins):
  
  # si el área de la moneada coin_i corresponde a moneda pequeña

    # agregar mask a small_coins (sume las imágenes)

  # else

    # agregar mask a big_coins (sume las imágenes)

# ---
# visualizar resultados
fig = plt.figure(figsize=(10, 10))
plt.imshow(small_coins, cmap='gray')
plt.axis('off')

fig = plt.figure(figsize=(10, 10))
plt.imshow(big_coins, cmap='gray')
plt.axis('off')

